# Анализ текста

Обработка естественного языка (Natural Language Processing, NLP) — это отрасль искусственного интеллекта (ИИ), которая имеет дело с письменным и устным языком. NLP можно использовать для создания решений, которые извлекают семантическое значение из текста или речи или формулируют осмысленные ответы на естественном языке.

*Когнитивные службы* Microsoft Azure включают службу *Анализ текста*, которая предоставляет некоторые нестандартные возможности NLP, включая определение ключевых фраз в тексте, а также классификацию текста на основе тональности.

![Робот читает записную книжку](./images/NLP.jpg)

Предположим, вымышленная организация *Margie's Travel* побуждает клиентов подавать отзывы о пребывании в отеле. Вы можете воспользоваться службой «Анализ текста» для подведения итогов отзывов путем извлечения ключевых фраз, определения того, какие отзывы положительны, а какие отрицательны, или анализа текста отзыва на предмет упоминания известных сущностей, таких как местоположение или люди.

## Просмотр документов отзывов

Начнем с того, что посмотрим на некоторые отзывы об отелях, которые оставили клиенты.

Отзывы находятся в текстовых файлах. Чтобы посмотреть их, просто выполните код в расположенной ниже ячейке с кодом, нажав на кнопку **Выполнить код в ячейке** (&#9655;) слева от ячейки.

In [ ]:
import os

# Read the reviews in the /data/reviews folder
reviews_folder = os.path.join('data', 'text', 'reviews')

# Create a collection of reviews with id (file name) and text (contents) properties
reviews = []
for file_name in os.listdir(reviews_folder):
    review_text = open(os.path.join(reviews_folder, file_name)).read()
    review = {"id": file_name, "text": review_text}
    reviews.append(review)

for review_num in range(len(reviews)):
    # print the review text
    print('{}\n{}\n'.format(reviews[review_num]['id'], reviews[review_num]['text']))

## Создание ресурса Cognitive Services

Для анализа текста в этих обзорах можно воспользоваться когнитивной службой **Анализ текста**. Для этого вам нужно предоставить ресурс службы **Анализ текста** или когнитивных служб **Cognitive Services** в вашей подписке Azure (Используйте ресурс службы «Анализ текста», если это единственная служба, которую вы планируете использовать, либо вы хотите отслеживать ее использование отдельно; в противном случае можно использовать ресурс Cognitive Services, чтобы объединить службу «Анализ текста» с другими когнитивными службами, позволяя разработчикам использовать единые конечную точку и ключ для доступа к ним).

Если у вас нет такого ресурса, воспользуйтесь следующими пошаговыми инструкциями для создания ресурса **Cognitive Services** в вашей подписке Azure:

> **Примечание**. Если у вас уже есть ресурс Cognitive Services, просто откройте его страницу **Быстрый запуск** и скопируйте его ключ и конечную точку в ячейку ниже. В противном случае следуйте приведенным ниже действиям для создания этого ресурса.

1. На другой вкладке браузера откройте портал Azure по адресу https://portal.azure.com, выполнив вход под своей учетной записью Microsoft.
2. Нажмите кнопку **&#65291;Создать ресурс**, выполните поиск по строке *Cognitive Services* и создайте ресурс **Cognitive Services** со следующими настройками:
    - **Подписка**. *Ваша подписка Azure*.
    - **Группа ресурсов**. *Выберите или создайте группу ресурсов с уникальным именем*.
    - **Регион**. *Выберите любой доступный регион:*
    - **Имя**. *Введите уникальное имя*.
    - **Ценовая категория**. S0
    - **Подтверждаю, что прочитал и понял уведомления**. Выбрано.
3. Дождитесь завершения развертывания. Затем перейдите на свой ресурс Cognitive Services и на странице **Обзор** щелкните ссылку для управления ключами службы. Для подключения к вашему ресурсу когнитивных служб из клиентских приложений вам понадобятся конечная точка и ключи.

### Получение ключа и конечной точки для ресурса Cognitive Services

Для использования ресурса когнитивных служб клиентским приложениям необходимы их конечная точка и ключ аутентификации:

1. На портале Azure откройте страницу **Ключи и конечная точка** для вашего ресурса Cognitive Service, скопируйте **Ключ1** для вашего ресурса и вставьте его в приведенный ниже код, заменив подстановочный текст **YOUR_COG_KEY**.
2. Скопируйте **конечную точку** для своего ресурса и вставьте ее в код ниже, заменив **YOUR_COG_ENDPOINT**.
3. Выполните код в расположенной ниже ячейке с кодом, нажав на зеленую кнопку <span style="color:green">&#9655;</span>.

In [ ]:
cog_key = 'YOUR_COG_KEY'
cog_endpoint = 'YOUR_COG_ENDPOINT'

print('Ready to use cognitive services at {} using key {}'.format(cog_endpoint, cog_key))

## Определение языка
Начнем с определения языка, на котором написаны эти отзывы.

In [ ]:
import os
from azure.cognitiveservices.language.textanalytics import TextAnalyticsClient
from msrest.authentication import CognitiveServicesCredentials

# Get a client for your text analytics cognitive service resource
text_analytics_client = TextAnalyticsClient(endpoint=cog_endpoint,
                                            credentials=CognitiveServicesCredentials(cog_key))

# Analyze the reviews you read from the /data/reviews folder earlier
language_analysis = text_analytics_client.detect_language(documents=reviews)

# print detected language details for each review
for review_num in range(len(reviews)):
    # print the review id
    print(reviews[review_num]['id'])

    # Get the language details for this review
    lang = language_analysis.documents[review_num].detected_languages[0]
    print(' - Language: {}\n - Code: {}\n - Score: {}\n'.format(lang.name, lang.iso6391_name, lang.score))

    # Add the detected language code to the collection of reviews (so we can do further analysis)
    reviews[review_num]["language"] = lang.iso6391_name

## Извлечение ключевых фраз

Теперь вы можете анализировать текст в отзывах клиентов для выявления ключевых фраз, которые дают некоторое представление об основных тезисах.

In [ ]:
# # Use the client and reviews you created in the previous code cell to get key phrases
key_phrase_analysis = text_analytics_client.key_phrases(documents=reviews)

# print key phrases for each review
for review_num in range(len(reviews)):
    # print the review id
    print(reviews[review_num]['id'])

    # Get the key phrases in this review
    print('\nKey Phrases:')
    key_phrases = key_phrase_analysis.documents[review_num].key_phrases
    # Print each key phrase
    for key_phrase in key_phrases:
        print('\t', key_phrase)
    print('\n')

Ключевые фразы могут помочь вам понять наиболее важные тезисы в каждом отзыве. Например, отзыв, содержащий фразу «предупредительный персонал» или «плохое обслуживание», может дать указание на некоторые из основных проблем рецензента.

## Определение тональности

Может быть полезно классифицировать отзывы как *положительные* или *отрицательные* на основе *оценки тональности*. И для этого можно использовать службу «Анализ текста».

In [ ]:
# Use the client and reviews you created previously to get sentiment scores
sentiment_analysis = text_analytics_client.sentiment(documents=reviews)

# Print the results for each review
for review_num in range(len(reviews)):

    # Get the sentiment score for this review
    sentiment_score = sentiment_analysis.documents[review_num].score

    # classifiy 'positive' if more than 0.5, 
    if sentiment_score < 0.5:
        sentiment = 'negative'
    else:
        sentiment = 'positive'

    # print file name and sentiment
    print('{} : {} ({})'.format(reviews[review_num]['id'], sentiment, sentiment_score))

## Извлечение известных сущностей

*Сущности* — это вещи, которые могут быть упомянуты в тексте и относятся к каким-то общеизвестным элементам. Например, людям, местам или датам. Предположим, вас интересуют даты и места, упомянутые в отзывах, — вы можете воспользоваться следующим кодом, чтобы найти их.

In [ ]:
# Use the client and reviews you created previously to get named entities
entity_analysis = text_analytics_client.entities(documents=reviews)

# Print the results for each review
for review_num in range(len(reviews)):
    print(reviews[review_num]['id'])
    # Get the named entitites in this review
    entities = entity_analysis.documents[review_num].entities
    for entity in entities:
        # Only print datetime or location entitites
        if entity.type in ['DateTime','Location']:
            link = '(' + entity.wikipedia_url + ')' if entity.wikipedia_id is not None else ''
            print(' - {}: {} {}'.format(entity.type, entity.name, link))

Обратите внимание, что некоторые объекты достаточно известны, чтобы иметь соответствующую страницу в Википедии, и в этом случае служба «Анализ текста» возвращает URL этой страницы.

## Дополнительные сведения

Дополнительные сведения о службе «Анализ текста» см. в [документации по службе «Анализ текста»](https://docs.microsoft.com/azure/cognitive-services/text-analytics/)